In [14]:
%pip install langchain langchain-core langgraph langchain-google-genai duckduckgo-search python-dotenv requests pydantic


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install langchain-community

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 1.8 MB/s eta 0:00:02
   ------------ --------------------------- 0.8/2.5 MB 1.5 MB/s eta 0:00:02
   ---------------- ----------------------- 1.0/2.5 MB 1.2 MB/s eta 0:00:02
   -------------------- ------------------- 1.3/2.5 MB 1.2 MB/s eta 0:00:02
   --------------------------------- ------ 2.1/2.5 MB 1.7 MB/s eta 0:00:01
   ------------------------------------- -- 2.4/2.5 MB 1.8 MB/s eta 0:00:01
   ------------------------------------- -- 2.4/2.5 MB 1.8 MB/s eta 0:00:01
   ------------------------------------- -- 2.4/2.5 MB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 1.2 MB/s  0:00:02
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.3/1.0 MB ? eta -:--:--
   ------------------------------ ---------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install duckduckgo-search

   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.5/4.0 MB 7.5 MB/s eta 0:00:01
   ------- -------------------------------- 0.8/4.0 MB 1.8 MB/s eta 0:00:02
   ------------- -------------------------- 1.3/4.0 MB 1.9 MB/s eta 0:00:02
   --------------- ------------------------ 1.6/4.0 MB 2.0 MB/s eta 0:00:02
   -------------------- ------------------- 2.1/4.0 MB 1.9 MB/s eta 0:00:02
   -------------------------- ------------- 2.6/4.0 MB 1.9 MB/s eta 0:00:01
   ------------------------------- -------- 3.1/4.0 MB 2.0 MB/s eta 0:00:01
   --------------------------------- ------ 3.4/4.0 MB 2.0 MB/s eta 0:00:01
   ---------------------------------------  3.9/4.0 MB 2.0 MB/s eta 0:00:01
   ---------------------------------------  3.9/4.0 MB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 4.0/4.0 MB 1.7 MB/s  0:00:02
   --------------------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
import os
from dotenv import load_dotenv
from duckduckgo_search import DDGS
from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
import requests
from typing import Optional, List
from langchain_core.language_models import LLM
from pydantic import Field


In [16]:
# Load from .env if available
load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

GEMINI_URL = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent"

In [19]:
# ✅ GeminiFlashLLM class that works with LangChain
class GeminiFlashLLM(LLM):
    api_key: str = Field(...)
    model_url: str = Field(...)

    @property
    def _llm_type(self) -> str:
        return "gemini-flash"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        headers = {"Content-Type": "application/json"}

        system_prompt = f"""
You are a smart agent. You can answer factual questions or use tools when needed.

If a tool is required:
Thought: I need to use a tool
Action: <tool name>
Action Input: <input>

Otherwise:
Final Answer: <answer>

Now answer:
Question: {prompt}
"""
        data = {
            "contents": [{"parts": [{"text": system_prompt}]}]
        }

        response = requests.post(
            f"{self.model_url}?key={self.api_key}",
            headers=headers,
            json=data
        )

        if response.status_code == 200:
            try:
                return response.json()['candidates'][0]['content']['parts'][0]['text'].strip()
            except Exception:
                return "⚠️ Error parsing Gemini response."
        else:
            return f"❌ API Error {response.status_code}: {response.text}"

In [24]:
def react_agent(query: str):
    print(f"\n🧠 Thinking about: {query}")
    response = llm.invoke(query)   # ✅ Correct call

    if "Action:" in response:
        try:
            action = response.split("Action:")[1].split("\n")[0].strip()
            action_input = response.split("Action Input:")[1].split("\n")[0].strip()

            if action.lower() == "calculator":
                result = calc_tool(action_input)
            elif action.lower() == "duckduckgo search":
                result = duckduckgo_search(action_input)
            else:
                result = "Unknown tool."

            print(f"🛠 Used {action} → {result}")
            final_prompt = f"Observation: {result}\nNow give Final Answer:"
            return llm.invoke(final_prompt)
        except Exception:
            return "⚠️ Error: Could not parse action."
    else:
        return response

In [25]:
user_input = input("🧠 Ask your question: ")
result = react_agent(user_input)
print("🤖 Gemini Agent:", result)


🧠 Thinking about: What is the capital of France?
🤖 Gemini Agent: Final Answer: The capital of France is Paris.


In [26]:
with open("gemini_log.txt", "a", encoding="utf-8") as f:
    f.write(f"User: {user_input}\n")
    f.write(f"Gemini Agent: {result}\n")
    f.write("="*60 + "\n")